<a href="https://colab.research.google.com/github/RicardoHernandezRodriguez/Hackated/blob/main/WebScrapingPeriodicosExpecificos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime
from typing import List, Dict
import re
from urllib.parse import urljoin, urlparse

class ScrapingPeriodicos:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'es-MX,es;q=0.8,en;q=0.5,en-US;q=0.3',
            'Accept-Encoding': 'gzip, deflate',
            'Connection': 'keep-alive',
        }
        self.session = requests.Session()
        self.session.headers.update(self.headers)

    def scrape_el_universal(self) -> List[Dict]:
        """Scraping específico para El Universal"""
        print("🔍 Buscando en El Universal...")
        noticias = []

        try:
            # Múltiples secciones relevantes
            urls_secciones = [
                'https://www.eluniversal.com.mx/cartera',  # Economía
                'https://www.eluniversal.com.mx/mundo',    # Internacional
                'https://www.eluniversal.com.mx/nacion'    # Nacional
            ]

            for url_seccion in urls_secciones:
                try:
                    response = self.session.get(url_seccion, timeout=10)
                    response.raise_for_status()
                    soup = BeautifulSoup(response.content, 'html.parser')

                    # El Universal usa diferentes estructuras, probamos varias
                    selectores = [
                        'article.story-item',
                        'div.story-item',
                        'article[class*="story"]',
                        'div[class*="nota"]',
                        'div.field-items div.field-item'
                    ]

                    articulos_encontrados = []
                    for selector in selectores:
                        articulos = soup.select(selector)
                        if articulos:
                            articulos_encontrados = articulos[:15]  # Limitar a 15 por sección
                            break

                    for articulo in articulos_encontrados:
                        titulo_elem = (articulo.find('h2') or
                                     articulo.find('h3') or
                                     articulo.find('h1') or
                                     articulo.find('a', class_='story-link'))

                        if titulo_elem:
                            titulo = titulo_elem.get_text().strip()

                            # Buscar el enlace
                            link_elem = titulo_elem.find('a') if titulo_elem.name != 'a' else titulo_elem
                            if not link_elem:
                                link_elem = articulo.find('a')

                            if link_elem and link_elem.get('href'):
                                url_completa = urljoin('https://www.eluniversal.com.mx', link_elem['href'])

                                # Buscar descripción
                                desc_elem = (articulo.find('p') or
                                           articulo.find('div', class_='summary') or
                                           articulo.find('div', class_='description'))
                                descripcion = desc_elem.get_text().strip()[:200] if desc_elem else ''

                                # Verificar relevancia
                                if self._es_relevante_para_tema(titulo, descripcion):
                                    noticia = {
                                        'titulo': titulo,
                                        'descripcion': descripcion,
                                        'url': url_completa,
                                        'fuente': 'El Universal',
                                        'fecha_publicacion': datetime.now().isoformat(),
                                        'seccion': url_seccion.split('/')[-1]
                                    }
                                    noticias.append(noticia)

                except Exception as e:
                    print(f"❌ Error en sección {url_seccion}: {e}")
                    continue

                time.sleep(1)  # Pausa entre secciones

            print(f"✅ El Universal: {len(noticias)} noticias encontradas")

        except Exception as e:
            print(f"❌ Error general en El Universal: {e}")

        return noticias

    def scrape_el_economista(self) -> List[Dict]:
        """Scraping específico para El Economista"""
        print("🔍 Buscando en El Economista...")
        noticias = []

        try:
            # Secciones relevantes de El Economista
            urls_secciones = [
                'https://www.eleconomista.com.mx/mercados',
                'https://www.eleconomista.com.mx/economia',
                'https://www.eleconomista.com.mx/internacionales',
                'https://www.eleconomista.com.mx/politica'
            ]

            for url_seccion in urls_secciones:
                try:
                    response = self.session.get(url_seccion, timeout=10)
                    response.raise_for_status()
                    soup = BeautifulSoup(response.content, 'html.parser')

                    # Selectores específicos para El Economista
                    selectores = [
                        'div.story-item',
                        'article.story',
                        'div.nota-item',
                        'div[class*="headline"]',
                        'li.story-item'
                    ]

                    articulos_encontrados = []
                    for selector in selectores:
                        articulos = soup.select(selector)
                        if articulos:
                            articulos_encontrados = articulos[:20]
                            break

                    # Si no encuentra con selectores, busca enlaces de noticias
                    if not articulos_encontrados:
                        enlaces_noticias = soup.find_all('a', href=re.compile(r'/[0-9]{4}/[0-9]{2}/[0-9]{2}/'))
                        articulos_encontrados = enlaces_noticias[:15]

                    for articulo in articulos_encontrados:
                        if articulo.name == 'a':
                            # Si el elemento ya es un enlace
                            titulo = articulo.get_text().strip()
                            url_completa = urljoin('https://www.eleconomista.com.mx', articulo['href'])
                            descripcion = ''
                        else:
                            # Buscar título dentro del artículo
                            titulo_elem = (articulo.find('h2') or
                                         articulo.find('h3') or
                                         articulo.find('h1') or
                                         articulo.find('a'))

                            if not titulo_elem:
                                continue

                            titulo = titulo_elem.get_text().strip()

                            # Buscar enlace
                            link_elem = titulo_elem if titulo_elem.name == 'a' else articulo.find('a')
                            if not link_elem or not link_elem.get('href'):
                                continue

                            url_completa = urljoin('https://www.eleconomista.com.mx', link_elem['href'])

                            # Buscar descripción
                            desc_elem = articulo.find('p', class_='summary') or articulo.find('p')
                            descripcion = desc_elem.get_text().strip()[:200] if desc_elem else ''

                        # Verificar relevancia y evitar duplicados
                        if (titulo and len(titulo) > 10 and
                            self._es_relevante_para_tema(titulo, descripcion) and
                            not any(n['url'] == url_completa for n in noticias)):

                            noticia = {
                                'titulo': titulo,
                                'descripcion': descripcion,
                                'url': url_completa,
                                'fuente': 'El Economista',
                                'fecha_publicacion': datetime.now().isoformat(),
                                'seccion': url_seccion.split('/')[-1]
                            }
                            noticias.append(noticia)

                except Exception as e:
                    print(f"❌ Error en sección {url_seccion}: {e}")
                    continue

                time.sleep(1)  # Pausa entre secciones

            print(f"✅ El Economista: {len(noticias)} noticias encontradas")

        except Exception as e:
            print(f"❌ Error general en El Economista: {e}")

        return noticias

    def scrape_milenio_economia(self) -> List[Dict]:
        """Scraping específico para la sección de economía de Milenio"""
        print("🔍 Buscando en Milenio...")
        noticias = []

        try:
            url = 'https://www.milenio.com/negocios'
            response = self.session.get(url, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            # Milenio tiene una estructura más consistente
            articulos = soup.find_all(['article', 'div'], class_=re.compile(r'(story|nota|news)'))[:15]

            for articulo in articulos:
                titulo_elem = (articulo.find('h2') or
                             articulo.find('h3') or
                             articulo.find('h1'))

                if titulo_elem:
                    # Obtener texto del título
                    link_elem = titulo_elem.find('a')
                    if link_elem:
                        titulo = link_elem.get_text().strip()
                        url_completa = urljoin('https://www.milenio.com', link_elem['href'])
                    else:
                        titulo = titulo_elem.get_text().strip()
                        link_elem = articulo.find('a')
                        if not link_elem:
                            continue
                        url_completa = urljoin('https://www.milenio.com', link_elem['href'])

                    # Buscar descripción
                    desc_elem = articulo.find('p')
                    descripcion = desc_elem.get_text().strip()[:200] if desc_elem else ''

                    if self._es_relevante_para_tema(titulo, descripcion):
                        noticia = {
                            'titulo': titulo,
                            'descripcion': descripcion,
                            'url': url_completa,
                            'fuente': 'Milenio',
                            'fecha_publicacion': datetime.now().isoformat(),
                            'seccion': 'negocios'
                        }
                        noticias.append(noticia)

            print(f"✅ Milenio: {len(noticias)} noticias encontradas")

        except Exception as e:
            print(f"❌ Error en Milenio: {e}")

        return noticias

    def extraer_contenido_completo(self, url: str, fuente: str) -> str:
        """Extrae el contenido completo de un artículo específico"""
        try:
            response = self.session.get(url, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            # Selectores comunes para el contenido del artículo
            selectores_contenido = [
                'div.story-body',
                'div.article-body',
                'div.content',
                'div.nota-content',
                'div[class*="body"]',
                'section.article-content',
                'div.entry-content'
            ]

            contenido = ""
            for selector in selectores_contenido:
                elemento = soup.select_one(selector)
                if elemento:
                    # Limpiar scripts y elementos no deseados
                    for script in elemento(["script", "style", "aside", "nav"]):
                        script.decompose()

                    paragrafos = elemento.find_all('p')
                    contenido = ' '.join([p.get_text().strip() for p in paragrafos if p.get_text().strip()])
                    break

            # Si no encuentra contenido estructurado, busca todos los párrafos
            if not contenido:
                paragrafos = soup.find_all('p')
                contenido = ' '.join([p.get_text().strip() for p in paragrafos if len(p.get_text().strip()) > 50])

            return contenido[:1000] if contenido else "No se pudo extraer contenido"

        except Exception as e:
            print(f"❌ Error extrayendo contenido de {url}: {e}")
            return "Error al extraer contenido"

    def buscar_en_todos_los_periodicos(self) -> List[Dict]:
        """Ejecuta scraping en todos los periódicos configurados"""
        print("🚀 Iniciando búsqueda en periódicos específicos...\n")

        todas_noticias = []

        # Ejecutar scraping en cada periódico
        periodicos = [
            self.scrape_el_universal,
            self.scrape_el_economista,
            self.scrape_milenio_economia
        ]

        for scraper in periodicos:
            try:
                noticias = scraper()
                todas_noticias.extend(noticias)
                print(f"✅ Completado: {scraper.__name__}")
                time.sleep(2)  # Pausa entre periódicos
            except Exception as e:
                print(f"❌ Error en {scraper.__name__}: {e}")

        # Eliminar duplicados por URL
        noticias_unicas = []
        urls_vistas = set()

        for noticia in todas_noticias:
            if noticia['url'] not in urls_vistas:
                noticias_unicas.append(noticia)
                urls_vistas.add(noticia['url'])

        print(f"\n📊 Total de noticias únicas: {len(noticias_unicas)}")
        return noticias_unicas

    def _es_relevante_para_tema(self, titulo: str, descripcion: str) -> bool:
        """Verifica si la noticia es relevante para el tema de interés"""
        texto_completo = (titulo + ' ' + descripcion).lower()

        # Palabras clave relacionadas con Trump/EE.UU. y economía
        keywords_politica = ['trump', 'estados unidos', 'eeuu', 'washington', 'biden', 'presidente']
        keywords_economia = ['mercado', 'economía', 'comercio', 'aranceles', 'dólar', 'peso', 'bolsa', 'inflación']
        keywords_mexico = ['méxico', 'mexico', 'mexicano', 'nacional']

        # Debe tener al menos una palabra de cada categoría O ser sobre México + economía
        tiene_politica = any(word in texto_completo for word in keywords_politica)
        tiene_economia = any(word in texto_completo for word in keywords_economia)
        tiene_mexico = any(word in texto_completo for word in keywords_mexico)

        return (tiene_politica and tiene_economia) or (tiene_mexico and tiene_economia)

    def buscar_con_terminos_especificos(self, terminos: List[str]) -> List[Dict]:
        """Busca noticias que contengan términos específicos"""
        print(f"🎯 Búsqueda específica con términos: {', '.join(terminos)}")

        todas_noticias = self.buscar_en_todos_los_periodicos()
        noticias_filtradas = []

        for noticia in todas_noticias:
            texto_completo = (noticia['titulo'] + ' ' + noticia['descripcion']).lower()

            # Verificar si contiene alguno de los términos específicos
            if any(termino.lower() in texto_completo for termino in terminos):
                noticias_filtradas.append(noticia)

        print(f"🔍 Noticias que contienen los términos: {len(noticias_filtradas)}")
        return noticias_filtradas

# Ejemplo de uso específico
if __name__ == "__main__":
    scraper = ScrapingPeriodicos()

    print("=== BÚSQUEDA EN PERIÓDICOS ESPECÍFICOS ===\n")

    # Opción 1: Buscar en todos los periódicos
    noticias_generales = scraper.buscar_en_todos_los_periodicos()

    # Opción 2: Buscar términos específicos
    terminos_especificos = ['Trump', 'aranceles', 'comercio México', 'peso mexicano', 'USMCA']
    noticias_especificas = scraper.buscar_con_terminos_especificos(terminos_especificos)

    # Mostrar resultados
    print("\n🏆 TOP 10 NOTICIAS MÁS RELEVANTES:")
    print("-" * 60)

    for i, noticia in enumerate(noticias_especificas[:10], 1):
        print(f"{i}. [{noticia['fuente']}] {noticia['titulo']}")
        print(f"   📄 {noticia['descripcion'][:100]}...")
        print(f"   🔗 {noticia['url']}")
        print(f"   📅 Sección: {noticia.get('seccion', 'N/A')}\n")

    # Extraer contenido completo de una noticia específica
    if noticias_especificas:
        print("📖 EXTRAYENDO CONTENIDO COMPLETO DE LA PRIMERA NOTICIA:")
        print("-" * 60)
        primera_noticia = noticias_especificas[0]
        contenido_completo = scraper.extraer_contenido_completo(
            primera_noticia['url'],
            primera_noticia['fuente']
        )
        print(f"Contenido: {contenido_completo[:500]}...")

=== BÚSQUEDA EN PERIÓDICOS ESPECÍFICOS ===

🚀 Iniciando búsqueda en periódicos específicos...

🔍 Buscando en El Universal...
✅ El Universal: 0 noticias encontradas
✅ Completado: scrape_el_universal
🔍 Buscando en El Economista...
✅ El Economista: 0 noticias encontradas
✅ Completado: scrape_el_economista
🔍 Buscando en Milenio...
✅ Milenio: 0 noticias encontradas
✅ Completado: scrape_milenio_economia

📊 Total de noticias únicas: 0
🎯 Búsqueda específica con términos: Trump, aranceles, comercio México, peso mexicano, USMCA
🚀 Iniciando búsqueda en periódicos específicos...

🔍 Buscando en El Universal...
✅ El Universal: 0 noticias encontradas
✅ Completado: scrape_el_universal
🔍 Buscando en El Economista...
✅ El Economista: 0 noticias encontradas
✅ Completado: scrape_el_economista
🔍 Buscando en Milenio...
✅ Milenio: 0 noticias encontradas
✅ Completado: scrape_milenio_economia

📊 Total de noticias únicas: 0
🔍 Noticias que contienen los términos: 0

🏆 TOP 10 NOTICIAS MÁS RELEVANTES:
------------